# 1. 真 - CoLLiE 快速上手

> 1.1 &ensp; CoLLiE 的 运行准备
> 
> 1.2 &ensp; 案例一：实现 Alpaca
> 
> 1.3 &ensp; CoLLiE 的 框架兼容

## 1.1 &ensp; CoLLiE 的 运行准备

&ensp; &ensp; 在上一节中，我们已经介绍了 CoLLiE 提出的背景、实现的功能 以及 包含的模块。通过上一节的叙述，想必大家已经可以配置好环境、成功地安装 CoLLiE，并且了解了 CoLLiE 的大致结构，遇到一些的功能需求可以找到对应的代码文件，尝试自行了解相关功能是如何实现的了。

&ensp; &ensp; 在这一节中，我们将介绍 **CoLLiE 的使用流程** 以及 **基础模块的使用**，包括 **Config**、**Dataset**、**Evaluator**、**Trainer**。而正所谓实践出真知，在详细介绍上述模块的使用细节与基础原理前，本小节将先通过一个 CoLLiE 的使用案例，告诉大家 CoLLiE 是如何使用、相关代码是如何运行起来的；接着在后续小节中，详细介绍各个部分代码的运行原理、对应模块的使用细节。

&ensp; &ensp; 多少显卡 多少模型 如何配置，按照2048长计算显卡需要数量，

| 模型-大小 | A800<br />全量参数微调 | A800<br />显存高效微调 | RTX 3090<br />全量参数微调 | RTX 3090<br />显存高效微调 |
|:----|:----:|:----:|:----:|:----:|
| LLaMA-7B | 3 | 1 | 8 | 1 |
| LLaMA-13B | 5 | 1 | 16 | 2 |
| LLaMA-30B | 12 | 1 | 48 | 4 |
| LLaMA-65B | 25 | 2 | 96 | 8 |
| LLaMA2-7B | 3 | 1 | 8 | 1 |
| LLaMA2-13B | 5 | 1 | 16 | 4 |
| LLaMA2-70B | 27 | 2 | 104 | 8 |
| IntermLM-7B | 3 | 1 | 8 | 1 |
| ChatGLM-6B | 3 | 1 | 8 | 1 |
| ChatGLM2-6B | 3 | 1 | 8 | 1 |

## 1.2 &ensp; 案例一：实现 Alpaca

&ensp; &ensp; 解决alpaca任务 能跑通 一个例子，一个使用CoLLie对LLaMA基座进行全参量Instruct Tuning，从而得到Alpaca的实例。

``` python
import sys
import json
import torch

from transformers import LlamaTokenizer
from transformers.generation.utils import GenerationConfig

from collie.config import CollieConfig

from collie.data import CollieDatasetForTraining
from collie.data import CollieDataLoader

from collie.controller.trainer import Trainer
from collie.controller.evaluator import EvaluatorForPerplexity, EvaluatorForGeneration

from collie.models.llama.model import LlamaForCausalLM
from collie.utils.monitor import StepTimeMonitor, TGSMonitor, MemoryMonitor, LossMonitor, EvalMonitor
from collie.metrics import DecodeMetric, PPLMetric, BleuMetric
from collie.module import GPTLMLoss
```

解释

``` python
# 1. 设置路径
# 1.1 预训练模型路径
pretrained_model = 'decapoda-research/llama-7b-hf'
# 1.2 数据集路径
data_path = 'alpaca.json'
# 1.3 Eval的decode结果保存路径
save_path = './result'
```

解释

``` python
# 2. 设置配置
# 2.1 加载配置
config = CollieConfig.from_pretrained(pretrained_model)
# 2.2 添加配置
config.tp_size = 2
config.dp_size = 2
config.pp_size = 1
config.train_epochs = 1
config.train_micro_batch_size = 8
config.eval_batch_size = 32
config.eval_per_n_steps = 100
config.ds_config = {
    "fp16": {"enabled": True},
    "monitor_config": {
        "enabled": True,
        "tag": "sophia_alpaca",
        "csv_monitor": {
            "enabled": True,
            "output_path": "./ds_logs/"
        }
    }
}
```

解释

``` python
# 3. 设置tokenizer
tokenizer = LlamaTokenizer.from_pretrained(pretrained_model, padding_side="left")

# 4. 加载数据集
dataset = CollieDatasetForTraining.from_json(data_path, tokenizer=tokenizer)
train_dataset = dataset[:-32]
eval_dataset = dataset[-32:]

# 5. 加载预训练模型
model = LlamaForCausalLM.from_config(config)
```

解释

``` python
# 6. 设置优化器
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 7. 添加监视器
monitors = [
    StepTimeMonitor(config),
    TGSMonitor(config),
    MemoryMonitor(config),
    LossMonitor(config),
    EvalMonitor(config)
]

# 8. 添加Evaluator
evaluator_ppl = EvaluatorForPerplexity(
    model = model,
    config = config,
    dataset = eval_dataset,
    monitors = [
        EvalMonitor(config)
    ],
    metrics = {
        'ppl': PPLMetric()
    }
)
evaluator_decode = EvaluatorForGeneration(
    model = model,
    config = config,
    tokenizer = tokenizer,
    dataset = eval_dataset,
    monitors = [
        EvalMonitor(config)
    ],
    metrics = {
        'decode': DecodeMetric(save_to_file = True, save_path = save_path)
    }

)
```

解释

``` python
# 9. 实例化trainer
trainer = Trainer(
    model = model,
    config = config,
    loss_fn = GPTLMLoss(-100),
    optimizer = optimizer,
    train_dataset = train_dataset,
    monitors = monitors,
    evaluators = [evaluator_ppl, evaluator_decode]
)

# 10. 训练/验证
trainer.train()

#  Command CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --rdzv_backend=c10d --rdzv_endpoint=localhost:29402 --nnodes=1 --nproc_per_node=4 train.py
```

## 1.3 &ensp; CoLLiE 的 框架兼容

&ensp; &ensp; 保存下来 Transformers加载（突出好用 能兼容 而不是全能）